# ezlocalai

Open this notebook in [Google Colab.](https://colab.research.google.com/)

Set Runtime Python 3 with T4 GPU or higher.

It takes several minutes to install dependencies.


In [ ]:
!apt install -y portaudio19-dev ffmpeg libportaudio2 libasound-dev
!pip install -qq scipy ftfy accelerate fastapi uvicorn pydantic requests tiktoken python-dotenv beautifulsoup4 faster-whisper pydub ffmpeg TTS sounddevice pyaudio webrtcvad pyngrok pdfplumber spacy python-multipart
!pip install -qq diffusers["torch"] transformers torch torchvision torchaudio
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!rm -rf sample_data .config
!git clone https://github.com/DevXT-LLC/ezlocalai .

## Start the server

Set the `NGROK_TOKEN` to use NGROK to expose your ezlocalai server to the public with as simple as an API key. [Get your free NGROK_TOKEN here.](https://dashboard.ngrok.com/get-started/your-authtoken)

You can see the ngrok URL in the cell output after running the cell.

Set the `LLM_TO_USE` with the name of the model from the models list. We will use `Mistral-7B-Instruct-v0.2` for this example since it is a smaller model that can still be useful.


In [ ]:
LLM_TO_USE = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
VISION_MODEL = "deepseek-ai/deepseek-vl-1.3b-chat"
GPU_LAYERS = 10
LLM_MAX_TOKENS = 16384

# Add your NGROK_TOKEN to your colab secrets if using Google Colab (Key logo on the left)
try:
    from google.colab import userdata
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    if not NGROK_TOKEN:
        raise
except:
    # If you're not using Google Colab, enter your NGROK_TOKEN below.
    NGROK_TOKEN = "Enter your ngrok token here"
if NGROK_TOKEN != "Enter your ngrok token here":
    with open('.env', 'r') as file:
        filedata = file.read()
    filedata = filedata.replace('NGROK_TOKEN=\n', f'NGROK_TOKEN={NGROK_TOKEN}\n')
    filedata = filedata.replace('DEFAULT_MODEL=TheBloke/phi-2-dpo-GGUF', f'DEFAULT_MODEL={LLM_TO_USE}')
    filedata = filedata.replace('GPU_LAYERS=0', f'GPU_LAYERS={GPU_LAYERS}')
    filedata = filedata.replace('LLM_MAX_TOKENS=0', f'LLM_MAX_TOKENS={LLM_MAX_TOKENS}')
    filedata = filedata.replace('VISION_MODEL=', f'VISION_MODEL={VISION_MODEL}')
    with open('.env', 'w') as file:
        file.write(filedata)
!uvicorn app:app --host 0.0.0.0 --port 8091 --workers 1 --proxy-headers